In [2]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence

import sympy as sp
import numpy as np

In [3]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
n_init, order, r = get_processed_and_shifted_recurrence(laplace2d)

In [4]:
def scale_recurrence(r):
    #We want to subsitute s(i) r^i_{ct} = g(i)
    g = sp.Function("g")
    s = sp.Function("s")
    n = sp.symbols("n")
    rct = sp.symbols("r_{ct}")

    r_new = r*rct**n
    for i in range(order):
        r_new = r_new.subs(s(n-i),g(n-i)/(rct**(n-i)))

    return r_new

In [5]:
r_new = scale_recurrence(r)

In [6]:
max_abs = .0000001
var = _make_sympy_vec("x", 2)
rct = sp.symbols("r_{ct}")
g = sp.Function("g")
n = sp.symbols("n")

In [7]:
r_new

(-1)**(n + 1)*r_{ct}**n*((-1)**(n - 3)*r_{ct}**(3 - n)*(n + (n - 2)**3 - 2*(n - 2)**2 - 2)*g(n - 3)/(x0**3 + x0*x1**2) + (-1)**(n - 2)*r_{ct}**(2 - n)*(-n + 3*(n - 2)**2 + 2)*g(n - 2)/(x0**2 + x1**2) + (-1)**(n - 1)*r_{ct}**(1 - n)*(3*x0**2*(n - 2) + x0**2 + x1**2*(n - 2) - x1**2)*g(n - 1)/(x0**3 + x0*x1**2))

In [8]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs
derivs = compute_derivatives(15)

In [9]:
def evaluate_recurrence(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        subs_dict[g(i)] = get_recurrence(recur.subs(rct, rct_val), i).subs(subs_dict).subs(coord_dict)
        print(get_recurrence(recur.subs(rct, 1),i))
    return np.array(list(subs_dict.values()))

In [10]:
def evaluate_recurrence_lamb(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(-2)] = 0
    subs_dict[g(-1)] = 0
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        exp = get_recurrence(recur.subs(rct, rct_val), i)
        f = sp.lambdify([var[0], var[1], g(i-1), g(i-2), g(i-3)], exp)
        subs_dict[g(i)] = f(coord_dict[var[0]], coord_dict[var[1]], subs_dict[g(i-1)],
                            subs_dict[g(i-2)], subs_dict[g(i-3)])
    subs_dict.pop(g(-2))
    subs_dict.pop(g(-1))
    return np.array(list(subs_dict.values()))

In [11]:
def evaluate_true(coord_dict, rct_val, p):
    retMe = []
    for i in range(p):
        exp = (derivs[i]*rct_val**i)
        f = sp.lambdify(var, exp)
        retMe.append(f(coord_dict[var[0]], coord_dict[var[1]]))
    return np.array(retMe)

In [22]:
def compute_error(pw):
    x_coord = 10**(-pw)
    y_coord = 1
    var = _make_sympy_vec("x", 2)
    coord_dict = {var[0]: x_coord, var[1]: y_coord}

    rct_val = x_coord
    exp = evaluate_recurrence_lamb(coord_dict, rct_val, r_new, 14)
    true = evaluate_true(coord_dict, rct_val, 14)
    print(exp)
    print(true)
    return np.abs(exp-true)/np.abs(true)

In [24]:
compute_error(3)

[4.99999750058881e-7 -9.99999000001000e-7 9.99997000005001e-7
 5.99998000004200e-12 -5.99994000021000e-12 -1.19999160001121e-16
 1.19997480020830e-16 5.03996236056665e-21 -5.03970436047030e-21
 3.22333878647354e-25 5.15931089723621e-24 3.83716481291881e-23
 3.45344442229625e-22 3.45344487898425e-21]
[ 4.99999750e-07 -9.99999000e-07  9.99997000e-07  5.99998000e-12
 -5.99994000e-12 -1.19999160e-16  1.19997480e-16  5.03993952e-21
 -5.03981856e-21 -3.62873347e-25  3.62860042e-25  3.99157622e-29
 -3.99136865e-29 -6.22680286e-33]


array([0, 2.11758448571812e-16, 4.23517744178265e-16,
       4.03898129797430e-16, 4.03900822467246e-16, 1.58609401692541e-11,
       4.75849247343332e-11, 4.53184684440781e-6, 2.26597825890807e-5,
       1.88828204410285, 13.2184597422124, 961314.681805742,
       8652282.26317474, 554609638070.892], dtype=object)

# Avoiding Cat Cancellation
The question is can we avoid catastrophic cancellation in the recurrence when $x_0 << 1$? Where $(x_0, y_0)$ is the location of the source?

If we formulate a recurrence for
$$
g(i, x_0, y_0) = \frac{d^i}{dx^i}|_{x = 0} G(x, y) r_{ct}^i
$$
we will inevitably get catastrophic cancellation when $x_0 << y_0$. Suppose we let $r_{ct} = x_0$ (we can scale up and down later with the true $r_{ct}$) and have
$$
g(n, x_0, y_0) = f_1(x_0, y_0, n-1) g(n-1, x_0, y_0) + f_2(x_0, y_0, n-2) g(n-2, x_0, y_0) + f_3(x_0, y_0, n-3) g(n-3, x_0, y_0)
$$
we could treat $g(n-1, x_0, y_0), g(n-2, x_0, y_0), g(n-3, x_0, y_0)$ as constants and taylor expand  $f_i(x_0, y_0, j)$ when $x_0  << y_0$. So instead we get for example:
$$
g(2) = -g(1) + \frac{4g(1)}{x_1^2} \frac{x_0^2}{2!} - \frac{48 g(1)}{x_1^4} \frac{x_0^4}{4!} 
$$
$$
g(3) = -\frac{4(g(1)-2g(2))}{x_1^2} \frac{x_0^2}{2!} - \frac{48 (g(1)-2g(2))}{x_1^4} \frac{x_0^4}{4!} 
$$
$$
g(4) = g(3) - \frac{4(g(1)-5g(2)+3g(3))}{x_1^2 }  \frac{x_0^2}{2!} - \frac{48(g(1)-5g(2)+3g(3))}{x_1^4} \frac{x_0^4}{4!} 
$$
$$
g(5) = 2g(4) + \frac{8(3g(2)-6g(3)+2g(4))}{x_1^2} \frac{x_0^2}{2!}
$$

In [ ]:
def generate_specialized_formula(i):
    a = sp.cancel(r_new.subs(rct, var[0]).subs(n, 3))
    for j in range(4):
        

In [75]:
a = sp.cancel(r_new.subs(rct, var[0]).subs(n, 3))
a

(-2*x0**2*g(1) + 4*x0**2*g(2))/(x0**2 + x1**2)

In [77]:
sp.simplify(sp.diff(a, var[0], 2).subs(var[0], 0))

4*(-g(1) + 2*g(2))/x1**2

In [17]:
'''
x_plot = [i for i in range(len(compute_error(0)))]
for i in range(1, 4):
    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))
plt.xlabel("order of derivative being computed")
plt.ylabel("absolute error")
plt.title("recurrence error vs order for different source-locations")
plt.legend(title='ratio of x_{coord_src}/y_{coord_src}')
plt.show()
'''

'\nx_plot = [i for i in range(len(compute_error(0)))]\nfor i in range(1, 4):\n    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))\nplt.xlabel("order of derivative being computed")\nplt.ylabel("absolute error")\nplt.title("recurrence error vs order for different source-locations")\nplt.legend(title=\'ratio of x_{coord_src}/y_{coord_src}\')\nplt.show()\n'